<a href="https://colab.research.google.com/github/DongDong-Zoez/pytorchAI/blob/main/GAN/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative Adversarial Network

GAN 是近幾年神經網路中最有趣的概念，我們不難從名子看出，他是個生成模型 (Generative)，且模型的學習是透過競爭 (Adversarial) 的方式

本篇筆記部分公式節錄自 [GAN](https://zhuanlan.zhihu.com/p/54096381?utm_id=0)



## Generative Model and Discriminant Model

在介紹生成模型與判別模型的差別之前，我們先來看看統計上的一些機率代號

- $p(x, y):$ 觀測值配上標籤一起出現的機率，也是我們在機器學習中蒐集到資料的機率
- $p(y|x):$ 給定一個觀測值，它屬於某一個標籤的機率，也是我們在機器學習中推斷的機率
- $p(y):$ 標籤的機率分配

判別模型是用大量的資料去學習做出判別，所以它計算的是 $p(y|x)$，而生成模型指的是我們在訓練時會專注在資料如何產生，或者資料的產生機制，所以我們會觀察 $p(x,y)$ 再計算 $p(y|x)$，這一步驟可以透過貝氏定理計算

$$
p(y|x) = \frac{p(x,y)}{p(x)}=\frac{p(x|y)p(y)}{\int p(x|y)p(y)dy} \propto p(x|y)p(y)
$$

至於為什麼要把分母 $p(x)$ 寫成 $\int p(x|y)p(y)dy$ ? 那是因為我們觀察到的資料是成對出現的 $(x,y)$，所以事實上我們有的是 $p(x|y)$

## Latent Space

不管是一張影像或者一段文字，他們其實都可以表示為一串高維度的向量，我們相信這個高維度的向量是從某一個機率分配 $p(x)$ 產生的，因為我們不知道這個分配是如何建構的，所以我們稱為 Latent (潛在)，我們的目的就是去估計 $p(x)$

### Approximate p(x)

我們用另一個分配 $G(z;\theta)$ ($\theta$ 為分配的參數) 去估計 $p(x)$，我們透過調整 $\theta$ 去找到最適合的分配形式，好了，那麼現在有兩個問題

1. 怎麼找? 怎麼更新參數?
2. 我們怎麼知道要哪個 $\theta$ 比較適合?

第一個問題其實很容易的可以用最大概似估計去解決 (ML, Maximum Likelihood)

第二個問題是透過 KL 散度計算，KL 散度是計算分配與分配之間的距離，當 KL 散度越小，表示 $p(x)$ 越接近 $G(z;\theta)$，也代表 $\theta$ 越適合 (當然也可以用其他散度，只要能夠衡量分配之間的距離即可)

#### Maximum Likelihood and KL Divergence

1. 我們從 $p(x)$ 蒐集到數據 $\{x_1, \cdots, x_n\}$
2. 代入 $G$ 後做相乘 (i.i.d.)，即 $\displaystyle L=\prod_{i=1}^nG(x_i;\theta)$
3. 因為 $\log$ 是單調函數，所以我們取對數方便計算，$\displaystyle\log L=\sum_{i=1}^n\log G(x_i;\theta)$
4. 當 $n$ sample 夠多時，$\displaystyle\log L\approx\mathbb{E}_{x\sim p(x)}[\log G(x;\theta)]$

所以我們的任務是找一個 $\theta^*$ 使得 $\theta^*=\arg\max_\theta\mathbb{E}_{x\sim p(x)}[\log G(x;\theta)]$

我們改寫一下上式 

$$
\theta^*=\arg\max_\theta\mathbb{E}_{x\sim p(x)}[\log G(x;\theta)]=\arg\max_\theta\mathbb{E}_{x\sim p(x)}[\log G(x;\theta)]-\mathbb{E}_{x\sim p(x)}[\log p(x)] = \arg\min_\theta \mbox{KL}(p||G)
$$

因為我們式中的變數為 $\theta$，所以第二個等號等於是原式加上一個常數，而第三個等號則是 KL 散度的定義

### Generator

在真實情況中，$p(x)$ 是一個高維度且非常複雜的分配，無法用一般統計模型去逼近，所以我們會採用神經網路來當作我們的 $G(z;\theta)$，此時，$\theta$ 為網路的參數，$z=(z_1,\cdots, z_m)$ 為 random noise，將 $x$ 代入網路的輸出收集起來，就是我們的 $G(z;\theta)$

所以我們等於是去找一個網路 $G^*(z;\theta^*)$ 使得

$$
G^* = \arg\min_G\mbox{KL}(p||G)
$$

### Discriminator

好吧，我們知道 $G$ 怎麼找了，但是我們如果不知道 $p(x)$ 和 $G(z;\theta)$ 的具體形式，我們能然沒辦法計算 KL 散度，因此，我們需要有一個模型幫我們判斷 $p(x)$ 和 $G(z;\theta)$ 是否足夠相近了，這個模型就是 Discriminator

注意到 $G(z;\theta)$ 是個隱分配 (Implicity distribution)

Note:

But what is exactly "Implicity distribution"?

- Implicity distribution: We do not know the analytic form of this probability distribution, but we can sample from it.

Actually, you can view generator as one of the implicity distribution, we do not know the explicity distribution, but we can generate images from noise or latent space

既然不能比較兩個機率分配的長相，那麼我們比較兩個機率分類的生成物呢? 這就是 Discriminator 的中心思想

- 我們可以從 $p(x)$ 中抽樣 (即觀測到的資料)
- 也可以從 $G(z;\theta)$ 中抽樣 (網路代入 random noise)

我們將兩者的輸出比較，且定義目標函數 $V(G,D)$ 如下

$$
V(G,D) = \mathbb{E}_{x\sim p(x)}[\log D(x)]+\mathbb{E}_{x\sim G(x;\theta)}[\log(1-D(x))] \\
D^*=\arg\max_DV(G,D)
$$

我們的任務是找一個 $D^*$ 最大化目標函數 $V(G,D)$，其等同於最大化 $\mathbb{E}_{x\sim p(x)}[\log D(x)]$ 或者最小化 $\mathbb{E}_{x\sim G(x;\theta)}[\log(1-D(x))]$

- maximum $\mathbb{E}_{x\sim p(x)}[\log D(x)]:$ 若輸入較可能是真實資料 ($p(x)$ 越高)，則 $D(x)$ 輸出盡量大
- minimum $\mathbb{E}_{x\sim G(x;\theta)}[\log(1-D(x))]:$ 若輸入可能是 Generator 生成出來的 ($G(x;\theta)$ 越高)，則 $D(x)$ 輸出盡量小

重新整理一下結論，$G$ 和 $D$ 的找法分別如下:

$$
D^*=\arg\max_DV(G,D) \\
G^*=\arg\min_G \mbox{Div}(p||G)=\arg\min_G\max_DV(G,D)
$$








### Algorithm

![](https://img-blog.csdnimg.cn/20200804144659118.jpg)

具體流程如下:

1. 從 Latent space 隨機抽樣 $m$ 個點
2. 從 Observed data 隨機抽樣 $m$ 個點
3. 固定生成器 $G$，利用梯度上升更新判別器 $D$
4. 固定判別器 $D$，利用梯度下降更新生成器 $G$ (D 為常數，故目標函數可省略)

重複上述過程直到收斂